# POS tagging using modified Viterbi

You need to accomplish the following in this assignment:

1. Write the vanilla Viterbi algorithm for assigning POS tags (i.e. without dealing with unknown words) 
2. Solve the problem of unknown words using at least two techniques. These techniques can use any of the approaches discussed in the class - lexicon, rule-based, probabilistic etc. Note that to implement these techniques, you can either write separate functions and call them from the main Viterbi algorithm, or modify the Viterbi algorithm, or both.
3. Compare the tagging accuracy after making these modifications with the vanilla Viterbi algorithm.
4. List down at least three cases from the sample test file (i.e. unknown word-tag pairs) which were incorrectly tagged by the original Viterbi POS tagger and got corrected after your modifications.

## Data Preparation

In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import defaultdict
from collections import Counter

Download treebank and universal_tagset from nltk if not downloaded already. Use - `nltk.download('treebank')` and `nltk.download('universal_tagset')`

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# lets have a look at the first 5 sentences
nltk_data[:5]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

In [4]:
# create a train and validation split of 95:5 for evaluation purpose
random.seed(100)
train_set, val_set = train_test_split(nltk_data,test_size=0.05,random_state=100)

print('Train set size: '+str(len(train_set)))
print('Validation set size: '+str(len(val_set)))

Train set size: 3718
Validation set size: 196


In [5]:
# have a look at first 5 train sentences
print(train_set[:5])

[[('One', 'NUM'), ('bright', 'ADJ'), ('sign', 'NOUN'), ('is', 'VERB'), ('that', 'ADP'), ('a', 'DET'), ('growing', 'VERB'), ('number', 'NOUN'), ('of', 'ADP'), ('women', 'NOUN'), ('have', 'VERB'), ('entered', 'VERB'), ('the', 'DET'), ('once', 'ADV'), ('male-dominated', 'ADJ'), ('field', 'NOUN'), (';', '.'), ('more', 'ADJ'), ('than', 'ADP'), ('a', 'DET'), ('third', 'ADJ'), ('of', 'ADP'), ('the', 'DET'), ('ringers', 'NOUN'), ('today', 'NOUN'), ('are', 'VERB'), ('women', 'NOUN'), ('.', '.')], [('``', '.'), ('These', 'DET'), ('days', 'NOUN'), (',', '.'), ('banking', 'NOUN'), ('customers', 'NOUN'), ('walk', 'VERB'), ('in', 'ADP'), ('the', 'DET'), ('door', 'NOUN'), ('*-1', 'X'), ('expecting', 'VERB'), ('you', 'PRON'), ('to', 'PRT'), ('have', 'VERB'), ('a', 'DET'), ('package', 'NOUN'), ('especially', 'ADV'), ('for', 'ADP'), ('them', 'PRON'), (',', '.'), ("''", '.'), ('Ms.', 'NOUN'), ('Moore', 'NOUN'), ('says', 'VERB'), ('*T*-2', 'X'), ('.', '.')], [('The', 'DET'), ('rights', 'NOUN'), (',', '.')

In [6]:
# getting list of tagged words i.e converting from sentence to words format
train_tagged_words = [tup for sent in train_set for tup in sent]
print('Total no. of tagged words: '+str(len(train_tagged_words)))

Total no. of tagged words: 95949


In [7]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
print('Sample tokens: '+str(tokens[:10]))
print('10 most common tokens: '+str(Counter(tokens).most_common(10)))

Sample tokens: ['One', 'bright', 'sign', 'is', 'that', 'a', 'growing', 'number', 'of', 'women']
10 most common tokens: [(',', 4650), ('the', 3855), ('.', 3641), ('of', 2216), ('to', 2050), ('a', 1798), ('in', 1501), ('and', 1446), ('*-1', 1064), ('0', 1044)]


In [8]:
# vocabulary - unique tokens
V = set(tokens)
print('Vocabulary size: '+str(len(V)))

Vocabulary size: 12106


In [9]:
# number of tags
tags = [pair[1] for pair in train_tagged_words]
T = sorted(set(tags),reverse=True)
print('No. of tags: '+str(len(T)))
print('Tags: '+str(T))
tags_counter = Counter(tags)
print('10 most common tags: '+str(tags_counter.most_common(10)))

No. of tags: 12
Tags: ['X', 'VERB', 'PRT', 'PRON', 'NUM', 'NOUN', 'DET', 'CONJ', 'ADV', 'ADP', 'ADJ', '.']
10 most common tags: [('NOUN', 27539), ('VERB', 12919), ('.', 11149), ('ADP', 9433), ('DET', 8318), ('X', 6301), ('ADJ', 6105), ('NUM', 3366), ('PRT', 3048), ('ADV', 3004)]


## Build the vanilla Viterbi based POS tagger

### Emission Probabilities

Computing P(w/t) and storing in V x T matrix

In [10]:
t = len(T)
v = len(V)

In [11]:
# compute word given tag: Emission Probability
tag_dict = {}
def word_given_tag(word, tag, train_bag = train_tagged_words):
    global tag_dict
    if tag in tag_dict:
        tag_list = tag_dict[tag]
    else:
        tag_list = [pair for pair in train_bag if pair[1]==tag]
        tag_dict[tag] = tag_list
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [12]:
%%time
# creating v x t emission matrix of tags
# each column is t, each row is w
# thus M(i, j) represents P(wj given ti)

words_tags_matrix = np.zeros((len(V), len(T)), dtype='float32')
for i, w in enumerate(list(V)):
    for j, t in enumerate(list(T)): 
        w_g_t = word_given_tag(w, t)
        words_tags_matrix[i, j] = w_g_t[0]/w_g_t[1]

Wall time: 3min 17s


In [13]:
# convert the matrix to a df for better readability
words_tags_df = pd.DataFrame(words_tags_matrix, columns=list(T), index=list(V))
words_tags_df.head()

,X,VERB,PRT,PRON,NUM,NOUN,DET,CONJ,ADV,ADP,ADJ,.
Aslacton,0.0,0.000000,0.0,0.0,0.0,0.000073,0.0,0.0,0.0,0.0,0.0,0.0
Form,0.0,0.000000,0.0,0.0,0.0,0.000036,0.0,0.0,0.0,0.0,0.0,0.0
print,0.0,0.000542,0.0,0.0,0.0,0.000036,0.0,0.0,0.0,0.0,0.0,0.0
advise,0.0,0.000077,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
assumption,0.0,0.000000,0.0,0.0,0.0,0.000109,0.0,0.0,0.0,0.0,0.0,0.0


### Transition Probabilities

Computing P(t/t) and storing in T x T matrix

In [14]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):    
    global tags_counter
    count_t1 = tags_counter[t1]
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [15]:
%%time
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        t2_g_t1 = t2_given_t1(t2, t1)
        tags_matrix[i, j] = t2_g_t1[0]/t2_g_t1[1]

Wall time: 1.61 s


In [16]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df.head()

,X,VERB,PRT,PRON,NUM,NOUN,DET,CONJ,ADV,ADP,ADJ,.
X,0.074433,0.204571,0.184891,0.055705,0.002857,0.062371,0.055229,0.010316,0.025393,0.144898,0.016505,0.162831
VERB,0.218438,0.168744,0.031427,0.035916,0.022448,0.111386,0.133292,0.005186,0.082050,0.091184,0.065640,0.034291
PRT,0.013123,0.405184,0.001969,0.017717,0.056102,0.245735,0.101050,0.002297,0.010171,0.019357,0.083661,0.043635
PRON,0.092720,0.484291,0.012261,0.007663,0.007280,0.211494,0.009195,0.004981,0.034100,0.023372,0.072031,0.040613
NUM,0.211824,0.016934,0.026144,0.001485,0.184195,0.352347,0.003862,0.013072,0.002674,0.035056,0.033571,0.118835


### Veterbi heuristics

In [17]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            #w_g_t = word_given_tag(words[key], tag)
            #emission_p = w_g_t[0]/w_g_t[1]
            emission_p = 0
            if word in words_tags_df.index:
                emission_p = words_tags_df.loc[word,tag]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [18]:
# list of untagged words from validation set
val_tagged_words = [tup[0] for sent in val_set for tup in sent]

In [19]:
%%time
# tagging the validation sentences
val_tagged_seq = Viterbi(val_tagged_words)

Wall time: 1.3 s


In [20]:
# accuracy
val_run_base = [tup for sent in val_set for tup in sent]
check = [i for i, j in zip(val_tagged_seq, val_run_base) if i == j]
accuracy = len(check)/len(val_tagged_seq)
print('Accuracy on validation set: '+str(accuracy))

Accuracy on validation set: 0.905013750793315


In [21]:
# storing y_valid and y_pred for later comparision
labels = sorted(T)
y_valid = [tup[1] for tup in val_run_base]
y_pred = [tup[1] for tup in val_tagged_seq]

Lets have a look at first 10 incorrectly tagged cases from validation set

In [22]:
incorrect_tagged_cases = [[val_run_base[i-1],j] for i, j in enumerate(zip(val_tagged_seq, val_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases[:10]

[[('to', 'PRT'), (('book', 'NOUN'), ('book', 'VERB'))],
 [('leaving', 'VERB'), (('stocks', 'NOUN'), ('stocks', 'ADV'))],
 [('stocks', 'ADV'), (('up', 'PRT'), ('up', 'ADP'))],
 [('carried', 'VERB'), (('over', 'ADP'), ('over', 'PRT'))],
 [('apparently', 'ADV'), (('ignored', 'X'), ('ignored', 'VERB'))],
 [('ignored', 'VERB'), (('reports', 'VERB'), ('reports', 'NOUN'))],
 [('Chilean', 'ADJ'), (('mine', 'NOUN'), ('mine', 'ADJ'))],
 [('the', 'DET'), (('Palestinian', 'ADJ'), ('Palestinian', 'NOUN'))],
 [('committee', 'NOUN'), (('first', 'ADJ'), ('first', 'ADV'))],
 [('.', '.'), (('Preston', 'X'), ('Preston', 'NOUN'))]]

As we can see all unknown words are marked as X because word's emmision probability is zero. Lets solve the problem at hand using smoothing, lexicon, rule and probability based methods.

## Solve the problem of unknown words

### 1. One-Count Smoothing

As described in the Appendix of <a href='http://www.cs.jhu.edu/~jason/465/hw-hmm/hw-hmm.pdf'>Jason Eisner's HMM tutorial</a>, the basic idea here is that for unknown words more probability mass should be given to tags that appear with a wider variety of low frequency words. For example, since the tag NOUN appears on a large number of different words and DETERMINER appears on a small number of different words, it is more likely that an unseen word will be a NOUN.

In [23]:
# find the most widely spread tag
print('Highest frequency tag: '+str(tags_counter.most_common(1)))

Highest frequency tag: [('NOUN', 27539)]


In [24]:
# lets find the highest frequency tag considering unique words
count_dict = defaultdict(set)
for tup in train_tagged_words:
    count_dict[tup[1]].add(tup[0])
for c in count_dict:
    count_dict[c] = len(count_dict[c])
print(count_dict)

defaultdict(<class 'set'>, {'NUM': 926, 'ADJ': 1772, 'NOUN': 6402, 'VERB': 2566, 'ADP': 118, 'DET': 46, 'ADV': 457, '.': 22, 'X': 443, 'PRON': 43, 'PRT': 21, 'CONJ': 18})


Even after considering only unique words, NOUN appears most number of times

In [25]:
# Modified Viterbi Heuristic
def Modified_Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            #w_g_t = word_given_tag(words[key], tag)
            #emission_p = w_g_t[0]/w_g_t[1]
            emission_p = 0
            if word in words_tags_df.index:
                emission_p = words_tags_df.loc[word,tag]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # handling unknown words
        if pmax == 0:
            state.append(tags_counter.most_common(1)[0][0])
        else:
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
            state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy of modified viterbi 1

In [26]:
%%time
# tagging the validation sentences
val_tagged_seq = Modified_Viterbi_1(val_tagged_words)

Wall time: 1.03 s


In [27]:
# accuracy
val_run_base = [tup for sent in val_set for tup in sent]
check = [i for i, j in zip(val_tagged_seq, val_run_base) if i == j]
accuracy = len(check)/len(val_tagged_seq)
print('Accuracy on validation set: '+str(accuracy))

Accuracy on validation set: 0.9348423947535435


***The accuracy has increased considerably (~3.5%) as compared to vanilla veterbi algorithm. This algorithm uses a better fallback estimation for unknown words.***

In [28]:
# storing y_valid and y_pred for later comparision
y_valid_1 = [tup[1] for tup in val_run_base]
y_pred_1 = [tup[1] for tup in val_tagged_seq]

### 2. Lexicon and Rule based solution

Making a Bigram + Unigram + Regex Tagger as a fallback tagger of vanilla viterbi algorithm to produce better estimation for unknown words.

In [29]:
# specify patterns for tagging
patterns = [
    (r'.*ly$', 'ADJ'),              # adjective
    (r'.*ful$', 'ADJ'),              # adjective
    (r'.*ish$', 'ADJ'),              # adjective
    (r'.*less$', 'ADJ'),              # adjective
    (r'.*st$', 'NUM'),              # number such as 1st
    (r'.*rd$', 'NUM'),              # number such as 3rd
    (r'.*th$', 'NUM'),              # number such as 11th
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*es$', 'VERB'),               # 3rd singular present
    (r'.*ould$', 'VERB'),              # modals
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                    # nouns
]

In [30]:
%%time
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

# lexicon backed up by the rule-based tagger
unigram_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)

bigram_tagger = nltk.BigramTagger(train_set, backoff=unigram_tagger)

Wall time: 3.68 s


In [31]:
# Viterbi Heuristic
def Modified_Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            #w_g_t = word_given_tag(words[key], tag)
            #emission_p = w_g_t[0]/w_g_t[1]
            emission_p = 0
            if word in words_tags_df.index:
                emission_p = words_tags_df.loc[word,tag]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # handling unknown words using backoff tagger
        if pmax == 0:
            state.append(bigram_tagger.tag([word])[0][1])
        else:
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
            state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy of modified viterbi 2

In [32]:
%%time
# tagging the validation sentences
val_tagged_seq = Modified_Viterbi_2(val_tagged_words)

Wall time: 1.21 s


In [33]:
# accuracy
val_run_base = [tup for sent in val_set for tup in sent]
check = [i for i, j in zip(val_tagged_seq, val_run_base) if i == j]
accuracy = len(check)/len(val_tagged_seq)
print('Accuracy on validation set: '+str(accuracy))

Accuracy on validation set: 0.945631478739158


***The accuracy has increased considerably (~4.5%) as compared to vanilla veterbi algorithm as well as modified viterbi 1. This algorithm uses a better fallback estimation for unknown words.***

In [34]:
# storing y_valid and y_pred for later comparision
y_valid_2 = [tup[1] for tup in val_run_base]
y_pred_2 = [tup[1] for tup in val_tagged_seq]

## Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

### Vanilla Viterbi Algorithm performance

In [35]:
# class-wise scores
print(classification_report(y_valid, y_pred, labels=labels, digits=3))

              precision    recall  f1-score   support

           .      1.000     1.000     1.000       566
         ADJ      0.874     0.740     0.801       292
         ADP      0.941     0.979     0.960       424
         ADV      0.919     0.820     0.867       167
        CONJ      1.000     1.000     1.000       108
         DET      0.982     0.956     0.969       407
        NOUN      0.969     0.852     0.907      1328
         NUM      0.993     0.806     0.890       180
        PRON      0.984     1.000     0.992       127
         PRT      0.943     0.971     0.957       171
        VERB      0.954     0.876     0.914       645
           X      0.512     1.000     0.678       312

    accuracy                          0.905      4727
   macro avg      0.923     0.917     0.911      4727
weighted avg      0.933     0.905     0.912      4727



### Viterbi algorithm with one-count smoothing

f1-score of `NOUN` as well as `X` tags have increased considerably due to better backoff estimation. 

In [36]:
# class-wise scores
print(classification_report(y_valid_1, y_pred_1, labels=labels, digits=3))

              precision    recall  f1-score   support

           .      1.000     1.000     1.000       566
         ADJ      0.878     0.736     0.801       292
         ADP      0.937     0.979     0.957       424
         ADV      0.919     0.820     0.867       167
        CONJ      1.000     1.000     1.000       108
         DET      0.982     0.956     0.969       407
        NOUN      0.869     0.968     0.916      1328
         NUM      0.993     0.806     0.890       180
        PRON      0.984     1.000     0.992       127
         PRT      0.949     0.971     0.960       171
        VERB      0.958     0.876     0.915       645
           X      1.000     0.962     0.980       312

    accuracy                          0.935      4727
   macro avg      0.956     0.923     0.937      4727
weighted avg      0.937     0.935     0.934      4727



### Viterbi Algorithm with lexicon and rule based tagger backoff

f1-score for `NOUN`, `VERB` and `NUM` has increased considerably due to regex and probabilistic backoff methods.

In [37]:
# class-wise scores
print(classification_report(y_valid_2, y_pred_2, labels=labels, digits=3))

              precision    recall  f1-score   support

           .      1.000     1.000     1.000       566
         ADJ      0.863     0.736     0.795       292
         ADP      0.937     0.979     0.957       424
         ADV      0.913     0.820     0.864       167
        CONJ      1.000     1.000     1.000       108
         DET      0.982     0.956     0.969       407
        NOUN      0.922     0.956     0.939      1328
         NUM      0.941     0.978     0.959       180
        PRON      0.984     1.000     0.992       127
         PRT      0.948     0.965     0.957       171
        VERB      0.929     0.933     0.931       645
           X      1.000     0.962     0.980       312

    accuracy                          0.946      4727
   macro avg      0.952     0.940     0.945      4727
weighted avg      0.945     0.946     0.945      4727



### Cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

Running the model on test data

In [38]:
test_set = []
with open('Test_sentences.txt') as test_file:
    for line in test_file:
        test_set.extend(word_tokenize(line))
print(test_set)

['Android', 'is', 'a', 'mobile', 'operating', 'system', 'developed', 'by', 'Google', '.', 'Android', 'has', 'been', 'the', 'best-selling', 'OS', 'worldwide', 'on', 'smartphones', 'since', '2011', 'and', 'on', 'tablets', 'since', '2013', '.', 'Google', 'and', 'Twitter', 'made', 'a', 'deal', 'in', '2015', 'that', 'gave', 'Google', 'access', 'to', 'Twitter', "'s", 'firehose', '.', 'Twitter', 'is', 'an', 'online', 'news', 'and', 'social', 'networking', 'service', 'on', 'which', 'users', 'post', 'and', 'interact', 'with', 'messages', 'known', 'as', 'tweets', '.', 'Before', 'entering', 'politics', ',', 'Donald', 'Trump', 'was', 'a', 'domineering', 'businessman', 'and', 'a', 'television', 'personality', '.', 'The', '2018', 'FIFA', 'World', 'Cup', 'is', 'the', '21st', 'FIFA', 'World', 'Cup', ',', 'an', 'international', 'football', 'tournament', 'contested', 'once', 'every', 'four', 'years', '.', 'This', 'is', 'the', 'first', 'World', 'Cup', 'to', 'be', 'held', 'in', 'Eastern', 'Europe', 'and',

In [39]:
%%time
# tagging the test sentences
test_tagged_seq = Viterbi(test_set)
print(test_tagged_seq)

[('Android', 'X'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'X'), ('.', '.'), ('Android', 'X'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'X'), ('worldwide', 'X'), ('on', 'ADP'), ('smartphones', 'X'), ('since', 'ADP'), ('2011', 'X'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'X'), ('.', '.'), ('Google', 'X'), ('and', 'CONJ'), ('Twitter', 'X'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'X'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'X'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'X'), ("'s", 'PRT'), ('firehose', 'X'), ('.', '.'), ('Twitter', 'X'), ('is', 'VERB'), ('an', 'DET'), ('online', 'X'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NOUN'), ('post', 'NOUN'), ('and', '

***Following are the highlighted tags which are corrected in subsequent methods:***
1. `('Android', 'X')`, ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), `('Google', 'X')`, ('.', '.'), `('Android', 'X')`, ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), `('OS', 'X')`, `('worldwide', 'X')`, ('on', 'ADP'), `('smartphones', 'X')`, ('since', 'ADP'), `('2011', 'X')`, ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), `('2013', 'X')`, ('.', '.')
<hr>
2. ('Donald', 'NOUN'), ('Trump', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), `('domineering', 'X')`, ('businessman', 'NOUN'), ('and', 'CONJ'), ('a', 'DET'), ('television', 'NOUN'), `('personality', 'X')`, ('.', '.')
<hr>
3. ('The', 'DET'), `('2018', 'X')`, `('FIFA', 'X')`, ('World', 'NOUN'), `('Cup', 'X')`, ('is', 'VERB'), ('the', 'DET'), `('21st', 'X')`, `('FIFA', 'X')`, ('World', 'NOUN'), `('Cup', 'X')`, (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), `('tournament', 'X')`, `('contested', 'X')`, ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.') 

In [40]:
%%time
# tagging the test sentences
test_tagged_seq = Modified_Viterbi_1(test_set)
print(test_tagged_seq)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET')

***Following are the highlighted tags which got corrected using this methods:***
1. `('Android', 'NOUN')`, ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), `('Google', 'NOUN')`, ('.', '.'), `('Android', 'NOUN')`, ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), `('OS', 'NOUN')`, `('worldwide', 'NOUN')`, ('on', 'ADP'), `('smartphones', 'NOUN')`, ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.')
<hr>
2. ('Donald', 'NOUN'), ('Trump', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), ('domineering', 'NOUN'), ('businessman', 'NOUN'), ('and', 'CONJ'), ('a', 'DET'), ('television', 'NOUN'), `('personality', 'NOUN')`, ('.', '.')
<hr>
3. ('The', 'DET'), ('2018', 'NOUN'), `('FIFA', 'NOUN')`, ('World', 'NOUN'), `('Cup', 'NOUN')`, ('is', 'VERB'), ('the', 'DET'), ('21st', 'NOUN'), `('FIFA', 'NOUN')`, ('World', 'NOUN'), `('Cup', 'NOUN')`, (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), `('tournament', 'NOUN')`, ('contested', 'NOUN'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')

In [41]:
%%time
# tagging the test sentences
test_tagged_seq = Modified_Viterbi_2(test_set)
print(test_tagged_seq)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'VERB'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), (

***Following are the highlighted tags which got corrected using this methods:***
1. `('Android', 'NOUN')`, ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), `('Google', 'NOUN')`, ('.', '.'), `('Android', 'NOUN')`, ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), `('OS', 'NOUN')`, `('worldwide', 'NOUN')`, ('on', 'ADP'), `('smartphones', 'NOUN')`, ('since', 'ADP'), `('2011', 'NUM')`, ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), `('2013', 'NUM')`, ('.', '.')
<hr>
2. ('Donald', 'NOUN'), ('Trump', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), `('domineering', 'VERB')`, ('businessman', 'NOUN'), ('and', 'CONJ'), ('a', 'DET'), ('television', 'NOUN'), `('personality', 'NOUN')`, ('.', '.')
<hr>
3. ('The', 'DET'), `('2018', 'NUM')`, `('FIFA', 'NOUN')`, ('World', 'NOUN'), `('Cup', 'NOUN')`, ('is', 'VERB'), ('the', 'DET'), `('21st', 'NUM')`, `('FIFA', 'NOUN')`, ('World', 'NOUN'), `('Cup', 'NOUN')`, (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), `('tournament', 'NOUN')`, `('contested', 'VERB')`, ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')